# Simulated Annealing and GSA for Feature Selection
## Simulated Annealing Algorithm) For Feature Selection on Churn Prediction
___

In [1]:
import sys
import os

# Get the directory of the current script or notebook
script_dir = os.path.dirname(os.path.abspath("__file__"))

# Construct the full path to the 'src' directory
src_dir = os.path.join(script_dir, '..', 'src')

# Append the 'src' directory to sys.path
sys.path.append(src_dir)

# Construct the full path to the 'data' directory
data_dir = os.path.join(script_dir, '..', 'data')

# Now, you can import the modules and access the data
from main import simulated_annealing
from GSA_implementation import GSA
from benchmarks import F1 

# Load your data (example)
import pandas as pd
df = pd.read_csv(os.path.join(data_dir, 'ChurnNormalize.csv'))


In [2]:
import sys
#sys.path.append(r'C:\Users\fakep\Documents\S3 ITS\Machine Learning with Python\Simulated-Annealing-Feature-Selection-main\src')
sys.path.append(r'C:\Users\hendr\Documents\S3 ITS\Machine Learning with Python\MachineLearning\Simulated-Annealing-Feature-Selection-main\src')

from main import simulated_annealing


In [3]:
import pandas as pd
import time  
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
from utils import train_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from GSA_implementation import GSA
from benchmarks import F1
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


#classifiers_name = "KNN"

 classifiers_name = {
     "Random Forest": RandomForestClassifier(),
     "SVM": SVC(),
     "Decision Tree": DecisionTreeClassifier(),
     "Logistic Regression": LogisticRegression(),
     "Naive Bayes": GaussianNB(),
     "KNN": KNeighborsClassifier(),
     "Gradient Boosting": GradientBoostingClassifier(),
}

IndentationError: unexpected indent (1106092380.py, line 22)

In [4]:
# Load data
#df = pd.read_csv(r'C:\Users\fakep\Documents\S3 ITS\Machine Learning with Python\Simulated-Annealing-Feature-Selection-main\data\churn2.csv')
#df = pd.read_csv(r'C:\Users\hendr\Documents\S3 ITS\Machine Learning with Python\MachineLearning\Simulated-Annealing-Feature-Selection-main\data\churn2.csv')
# Drop the first column
#df = df.iloc[:, 1:]

#dataset1
# Assuming the target variable is named 'target' and all others are features
#X_train = df.drop(columns=['churn'])
#y_train = df['churn']

#dataset2
X_train = df.drop(columns=['Churn'])
y_train = df['Churn']

In [5]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,0,1,0.013889,0,0,1,2,1,2,2,2,2,2,1,3,0.115423,0.003437,0
1,0,0,1,1,0.472222,1,1,1,1,2,1,2,2,2,1,0,2,0.385075,0.217564,0
2,0,0,1,1,0.027778,1,1,1,1,1,2,2,2,2,2,1,2,0.354229,0.012453,1
3,0,0,1,1,0.625000,0,0,1,1,2,1,1,2,2,1,0,1,0.239303,0.211951,0
4,1,0,1,1,0.027778,1,1,2,2,2,2,2,2,2,2,1,3,0.521891,0.017462,1


In [ ]:
import numpy as np
import random
import time

# Define the Adaptive GSA-SA function
def gsa_sa_iterative(X_train, y_train, max_iters=100, gsa_iters=10, sa_iters=50, alpha=0.93, beta=1, convergence_threshold=0.001):
    """
    A hybrid GSA-SA function where GSA is run at each SA iteration.
    This approach continuously reinitializes SA with GSA-generated solutions.
    """
    
    # Set up GSA parameters
    lb = [0] * X_train.shape[1]
    ub = [1] * X_train.shape[1]
    dim = len(lb)
    PopSize = 20  # Smaller population size for efficiency in each SA iteration

    # Initialize SA variables
    best_solution = None
    best_fitness = float('inf')  # Set to infinity to minimize
    results = []

    for sa_iter in range(sa_iters):
        print(f"Starting SA Iteration {sa_iter + 1}")
        
        # Run GSA to generate a starting solution for this SA iteration
        gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=gsa_iters, df=X_train)
        current_solution = gsa_solution.gBest
        current_fitness = gsa_solution.best_fitness

        # Initialize the solution with GSA output
        temperature = 2 * len(current_solution)  # Dynamic temperature based on feature count
        cooling_schedule = alpha

        # Run Simulated Annealing from the GSA-generated solution
        for sa_inner_iter in range(max_iters):
            start_time = time.time()

            # Generate a neighboring solution by tweaking the GSA result
            neighbor_solution = generate_neighbor(current_solution, lb, ub)
            neighbor_fitness = fitness_function(neighbor_solution, X_train, y_train)

            # Determine acceptance
            if neighbor_fitness < current_fitness or np.random.rand() < np.exp((current_fitness - neighbor_fitness) / temperature):
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness
                print(f"New solution accepted with fitness {current_fitness}")
            
            # Track the best solution found
            if current_fitness < best_fitness:
                best_solution = current_solution
                best_fitness = current_fitness

            # Cool down
            temperature *= cooling_schedule
            end_time = time.time()
            iteration_time = end_time - start_time

            # Store results for this iteration
            results.append({
                'SA Iteration': sa_iter + 1,
                'SA Inner Iteration': sa_inner_iter + 1,
                'Fitness': current_fitness,
                'Best Fitness': best_fitness,
                'Run Time (s)': iteration_time
            })

            # Convergence check
            if abs(best_fitness - current_fitness) < convergence_threshold:
                print(f"No significant improvement. Stopping at SA Inner Iteration {sa_inner_iter + 1}.")
                break

    print(f"Best solution found: {best_solution} with fitness: {best_fitness}")
    return best_solution, results


def generate_neighbor(solution, lb, ub):
    """
    Generates a neighboring solution by making small perturbations to the current solution.
    Ensures that the new solution respects the problem's bounds.
    """
    perturbation = np.random.uniform(-0.1, 0.1, len(solution))
    neighbor = np.clip(solution + perturbation, lb, ub)
    return neighbor


def fitness_function(solution, X_train, y_train):
    """
    Evaluates the fitness of a solution. This function should consider both accuracy and feature subset size.
    Replace this with your specific fitness function.
    """
    # Example fitness: inverse of accuracy for minimization, plus penalty for feature count
    accuracy = evaluate_model(X_train.iloc[:, solution > 0.5], y_train)
    feature_count = np.sum(solution > 0.5)
    return (1 - accuracy) + (feature_count / X_train.shape[1])

# Example usage:
# X_train, y_train = ... # Load your churn dataset
# best_solution, results = gsa_sa_iterative(X_train, y_train)


In [6]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

# Define the classifiers
classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
    "SVM": SVC(probability=True),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Initialize an empty DataFrame to store average and best metrics for each run
avg_and_best_results = pd.DataFrame()

# Number of runs
num_runs = 10

# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30
iters = 100

# Generate a timestamp for dynamic file names
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Get classifier names for the file name
classifier_names = "_".join(classifiers.keys())

# Loop through each classifier
for name, clf in classifiers.items():
    classifier_results = []
    
    for run in range(1, num_runs + 1):
        #print(f"Running FS-SA with {name} (Run {run})...")
        print(f"Running FS-GSA_SA with {name} (Run {run})...")
        
        # Run the simulated annealing process with the current classifier
        #results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name=name, run_index=run)
        
        # Run GSA to get the best solution
        gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=iters, df=df)
        initial_solution = gsa_solution.gBest

        # Run Simulated Annealing with the initial solution from GSA
        results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name = name, run_index = run, initial_solution=initial_solution)
        
        # Add columns for the classifier name and run number
        results['Classifier'] = name
        results['Run'] = run
        
        # Append the results to the combined DataFrame
        all_results = pd.concat([all_results, results], ignore_index=True)
        
        # Store the best metric from this run
        classifier_results.append(best_metric)
        
        # Calculate the average metrics for this run
        avg_metrics = results.mean()
        avg_metrics['Classifier'] = name
        avg_metrics['Run'] = run
        avg_metrics['Type'] = 'Average'

        # Store the best model's metrics
        best_metrics = pd.Series(best_metric)
        best_metrics['Classifier'] = name
        best_metrics['Run'] = run
        best_metrics['Type'] = 'Best Model'

        # Append to the DataFrame
        avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([avg_metrics]), pd.DataFrame([best_metrics])], ignore_index=True)

    # Calculate overall averages across all runs for this classifier
    overall_avg_metrics = avg_and_best_results[(avg_and_best_results['Classifier'] == name) & (avg_and_best_results['Type'] == 'Average')].mean(numeric_only=True)
    overall_avg_metrics['Classifier'] = name
    overall_avg_metrics['Run'] = 'Overall Average'
    overall_avg_metrics['Type'] = 'Average'

    overall_best_metrics = avg_and_best_results[(avg_and_best_results['Classifier'] == name) & (avg_and_best_results['Type'] == 'Best Model')].mean(numeric_only=True)
    overall_best_metrics['Classifier'] = name
    overall_best_metrics['Run'] = 'Overall Best'
    overall_best_metrics['Type'] = 'Best Model'

    # Append overall averages to the DataFrame
    avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([overall_avg_metrics]), pd.DataFrame([overall_best_metrics])], ignore_index=True)

# Dynamic file names
output_csv = f'fs_sa_results_{classifier_names}_{num_runs}runs_{timestamp}.csv'
average_output_csv = f'fs_sa_average_best_metrics_{classifier_names}_{num_runs}runs_{timestamp}.csv'

# Save the combined results to a CSV file
all_results.to_csv(output_csv, index=False)

# Save the average and best metrics to another CSV file
avg_and_best_results.to_csv(average_output_csv, index=False)

print(f"Detailed results saved to {output_csv}")
print(f"Average and best metrics saved to {average_output_csv}")


Running FS-GSA_SA with KNN (Run 1)...
GSA is optimizing  "F1"
['At iteration 1 the best fitness is 0.2745512740678009']
['At iteration 2 the best fitness is 0.280000249084614']
['At iteration 3 the best fitness is 0.3062378259894886']
['At iteration 4 the best fitness is 0.3142605923232122']
['At iteration 5 the best fitness is 0.3142605923232122']
['At iteration 6 the best fitness is 0.3994171420031384']
['At iteration 7 the best fitness is 0.3994171420031384']
['At iteration 8 the best fitness is 0.3994171420031384']
['At iteration 9 the best fitness is 0.3994171420031384']
['At iteration 10 the best fitness is 0.3994171420031384']
['At iteration 11 the best fitness is 0.3994171420031384']
['At iteration 12 the best fitness is 0.3994171420031384']
['At iteration 13 the best fitness is 0.3994171420031384']
['At iteration 14 the best fitness is 0.3994171420031384']
['At iteration 15 the best fitness is 0.3994171420031384']
['At iteration 16 the best fitness is 0.3994171420031384']
['At

Worse Fitness but accepted. Fitness change: 0.0084, Acceptance probability: 0.9991, Random number: 0.5327
Starting Iteration 22
Worse Fitness but accepted. Fitness change: 0.0090, Acceptance probability: 0.9989, Random number: 0.8317
Starting Iteration 23
Worse Fitness but accepted. Fitness change: 0.0084, Acceptance probability: 0.9989, Random number: 0.8540
Starting Iteration 24
Worse Fitness but accepted. Fitness change: 0.0129, Acceptance probability: 0.9982, Random number: 0.5250
Starting Iteration 25
Worse Fitness but accepted. Fitness change: 0.0124, Acceptance probability: 0.9981, Random number: 0.3964
Starting Iteration 26
Worse Fitness but accepted. Fitness change: 0.0109, Acceptance probability: 0.9982, Random number: 0.9105
Starting Iteration 27
Worse Fitness but accepted. Fitness change: 0.0094, Acceptance probability: 0.9984, Random number: 0.5896
Starting Iteration 28
Worse Fitness but accepted. Fitness change: 0.0109, Acceptance probability: 0.9980, Random number: 0.017

C:\Users\hendr\AppData\Local\Temp\ipykernel_501656\2273161487.py:71: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_metrics = results.mean()


['At iteration 1 the best fitness is 0.27666276434104664']
['At iteration 2 the best fitness is 0.27666276434104664']
['At iteration 3 the best fitness is 0.3099860512616136']
['At iteration 4 the best fitness is 0.33528657184845695']
['At iteration 5 the best fitness is 0.33528657184845695']
['At iteration 6 the best fitness is 0.33528657184845695']
['At iteration 7 the best fitness is 0.33528657184845695']
['At iteration 8 the best fitness is 0.33528657184845695']
['At iteration 9 the best fitness is 0.33528657184845695']
['At iteration 10 the best fitness is 0.33528657184845695']
['At iteration 11 the best fitness is 0.33528657184845695']
['At iteration 12 the best fitness is 0.3657409021844721']
['At iteration 13 the best fitness is 0.3657409021844721']
['At iteration 14 the best fitness is 0.3657409021844721']
['At iteration 15 the best fitness is 0.3657409021844721']
['At iteration 16 the best fitness is 0.3657409021844721']
['At iteration 17 the best fitness is 0.365740902184472

Improvement in Fitness from 0.2404 to 0.2324 - New subset accepted
Starting Iteration 22
Improvement in Fitness from 0.2324 to 0.2314 - New subset accepted
Starting Iteration 23
Improvement in Fitness from 0.2314 to 0.2309 - New subset accepted
Starting Iteration 24
Improvement in Fitness from 0.2309 to 0.2250 - New subset accepted
Starting Iteration 25
Improvement in Fitness from 0.2250 to 0.2225 - New subset accepted
Starting Iteration 26
Worse Fitness but accepted. Fitness change: 0.0079, Acceptance probability: 0.9987, Random number: 0.5114
Starting Iteration 27
Worse Fitness but accepted. Fitness change: 0.0015, Acceptance probability: 0.9997, Random number: 0.2381
Starting Iteration 28
Worse Fitness but accepted. Fitness change: 0.0009, Acceptance probability: 0.9998, Random number: 0.2306
Starting Iteration 29
Subset already visited
Worse Fitness but accepted. Fitness change: 0.0306, Acceptance probability: 0.9939, Random number: 0.5751
Starting Iteration 30
Worse Fitness but ac

...

C:\Users\hendr\AppData\Local\Temp\ipykernel_501656\2273161487.py:71: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_metrics = results.mean()


['At iteration 1 the best fitness is 0.2679918798415822']
['At iteration 2 the best fitness is 0.2743779111764267']
['At iteration 3 the best fitness is 0.28152140882257704']
['At iteration 4 the best fitness is 0.28152140882257704']
['At iteration 5 the best fitness is 0.28152140882257704']
['At iteration 6 the best fitness is 0.28152140882257704']
['At iteration 7 the best fitness is 0.28152140882257704']
['At iteration 8 the best fitness is 0.2842747901462126']
['At iteration 9 the best fitness is 0.2842747901462126']
['At iteration 10 the best fitness is 0.2842747901462126']
['At iteration 11 the best fitness is 0.2842747901462126']
['At iteration 12 the best fitness is 0.2842747901462126']
['At iteration 13 the best fitness is 0.2842747901462126']
['At iteration 14 the best fitness is 0.2842747901462126']
['At iteration 15 the best fitness is 0.2842747901462126']
['At iteration 16 the best fitness is 0.2842747901462126']
['At iteration 17 the best fitness is 0.2842747901462126']
[

C:\Users\hendr\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Starting Iteration 1
Worse Fitness but accepted. Fitness change: 0.0024, Acceptance probability: 0.9999, Random number: 0.3646
Starting Iteration 2
Improvement in Fitness from 0.2340 to 0.2037 - New subset accepted
Starting Iteration 3
Worse Fitness but accepted. Fitness change: 0.0005, Acceptance probability: 1.0000, Random number: 0.4730
Starting Iteration 4
Worse Fitness but accepted. Fitness change: 0.0001, Acceptance probability: 1.0000, Random number: 0.5496
Starting Iteration 5
Worse Fitness but accepted. Fitness change: 0.0006, Acceptance probability: 1.0000, Random number: 0.4363
Starting Iteration 6
Improvement in Fitness from 0.2037 to 0.2018 - New subset accepted
Starting Iteration 7
Worse Fitness but accepted. Fitness change: 0.0232, Acceptance probability: 0.9991, Random number: 0.8066
Starting Iteration 8
Worse Fitness but accepted. Fitness change: 0.0040, Acceptance probability: 0.9998, Random number: 0.5836
Starting Iteration 9
Worse Fitness but accepted. Fitness chang

C:\Users\hendr\AppData\Local\Temp\ipykernel_501656\2273161487.py:71: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_metrics = results.mean()


In [ ]:
results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name ="KNN")

In [ ]:
# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30
iters = 100

# Run GSA to get the best solution
gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=iters, df=df)
initial_solution = gsa_solution.gBest

# Run Simulated Annealing with the initial solution from GSA
results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, initial_solution=initial_solution)


In [ ]:
results

In [ ]:
best_metric

In [ ]:
best_subset_cols

In [ ]:
# Multiple Runs for Robustness Testing
num_runs = 10  # Number of runs
run_times = []
all_metrics = []
selected_features_count = []


classifier_name = "KNN"
fitness_function = "accuracy"


for run in range(num_runs):
    print(f"Starting Run {run + 1}")
    
    # Start the timer for the run
    start_time = time.time()
    
    # Run the simulated annealing process
    results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name, fitness_function, run_index=run+1)
    
    # End the timer for the run
    end_time = time.time()
    
    # Calculate the run time and store it
    run_time = end_time - start_time
    run_times.append(run_time)
    
    print(f"Run {run + 1} completed in {run_time:.2f} seconds")
    
    
    #store the best metric from this run
    all_metrics.append(best_metric)
    # Optionally store the results
    # all_results.append((results, best_metric, best_subset_cols))
    
    # Store the number of features selected in this run
    selected_features_count.append(len(best_subset_cols))
    

# After all runs are complete
mean_runtime = np.mean(run_times)
std_runtime = np.std(run_times)

print(f"Average run time over {num_runs} runs: {mean_runtime:.2f} seconds")
print(f"Standard deviation of run time: {std_runtime:.2f} seconds")

# Calculate average and standard deviation of each metric
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
std_metrics = {key: np.std([metric[key] for metric in all_metrics]) for key in all_metrics[0]}

# Display average and standard deviation of metrics
print("\nAverage Metrics over all runs:")
for metric, value in avg_metrics.items():
    print(f"{metric}: {value:.3f}")

print("\nStandard Deviation of Metrics over all runs:")
for metric, value in std_metrics.items():
    print(f"{metric}: {value:.3f}")

# Display the best metrics for each run
print("\nBest Metrics from each run:")
for run_index, metrics in enumerate(all_metrics):
    print(f"Run {run_index + 1}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.3f}")

# Calculate and display the average number of features selected
average_features_selected = np.mean(selected_features_count)
print(f"\nAverage number of features selected over {num_runs} runs: {average_features_selected:.2f}")


In [ ]:
results